In [1]:
from tools import *


2024-06-02 22:56:34,900 ib_async.client INFO Connecting to 127.0.0.1:4001 with clientId 32...
2024-06-02 22:56:34,908 ib_async.client INFO Connected
2024-06-02 22:56:35,001 ib_async.client INFO Logged on to server version 176
2024-06-02 22:56:35,024 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfuture
2024-06-02 22:56:35,032 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-06-02 22:56:35,046 ib_async.wrapper INFO Warning 2107, reqId -1: HMDS data farm connection is inactive but should be available upon demand.ushmds
2024-06-02 22:56:35,048 ib_async.wrapper INFO Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
2024-06-02 22:56:35,049 ib_async.client INFO API connection ready
2024-06-02 22:56:35,086 ib_async.wrapper INFO position: Position(account='U2340948', contract=Stock(conId=4762, symbol='BA', exchange='NYSE', currency='USD', localSymbol='BA', tradingClass='BA'), position=254.0, avgCost

Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)


2024-06-02 22:56:36,010 ib_async.wrapper INFO openOrder: Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=Order(permId=2098634708, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=18455.0, auxPrice=0.0, tif='GTC', ocaGroup='2098634707', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=0, status='PreSubmitted', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[], advancedError='')
2024-06-02 22:56:36,017 ib_async.wrapper INFO orderStatus: Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractM

2024-06-02 23:13:09,011 ib_async.wrapper INFO Warning 2103, reqId -1: Market data farm connection is broken:usfarm
2024-06-02 23:13:09,705 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-06-02 23:13:42,075 ib_async.wrapper INFO Warning 2103, reqId -1: Market data farm connection is broken:usfarm
2024-06-02 23:13:42,775 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-06-02 23:19:51,372 ib_async.wrapper INFO Warning 2103, reqId -1: Market data farm connection is broken:usfarm
2024-06-02 23:19:52,050 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-06-02 23:21:49,202 ib_async.wrapper ERROR Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
2024-06-02 23:21:49,474 ib_async.wrapper ERROR Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
2024-06-02 23:21:55,544 ib_async.wrapper ERROR Error 1100, r

In [2]:
open_permid = get_last_trade_permid(n=0)
close_permid = get_last_trade_permid()
print(open_permid, ";", close_permid)


1533812277 ; 1533812278


In [13]:
DEBUG = True
LIVE = False



def run_ss_2(strategy_details, open_permid = None, close_permid = None, push = False):
    open_trade = get_trade_by_permid(open_permid)
    close_trade = get_trade_by_permid(close_permid)

    open_order_ts = None

    push_notifications(
        f"{strategy_details} / {open_permid} / {close_permid}", push
    )

    push_notifications(f"OPEN TRADE:: {open_trade}", push)
    push_notifications(f"CLOSE TRADE:: {close_trade}", push)

    pause = False

    def print_summary():
        print_clear()
        print_line()
        print_orderbook()
        print_line()

        print_positions()
        print_line()

        print_openOrders()
        print_line()

        # openOrders = parse_ibrecords(ib.openOrders())
        # print_ibrecords_table(
        #     openOrders,
        #     cols=[
        #         "localSymbol",
        #         "permId",
        #         "status",
        #         "orderType",
        #         "action",
        #         "lmtPrice",
        #         "remaining",
        #     ],
        # )

        print_account_summary()
        print_line()

        print(f"------- {strategy_details['strategy']} / {strategy_details['open_ticks']}-{strategy_details['close_ticks']} / {strategy_details['pause_seconds']} seconds -------")
        print_line()

        print_order(open_trade)
        print_line()

        print_order(close_trade)
        print_line()

    while ib.sleep(1):
        print_summary()

        if open_order_ts is not None and (datetime.datetime.now() < open_order_ts > datetime.timedelta(seconds=strategy_details["pause_seconds"])):
            continue
            
        if DEBUG:
            proceed = input("Proceed? ")

            if proceed.lower() == "n":
                continue

        # execution here
        # first order of the strategy
        if open_trade is None and close_trade is None:
            action = strategy_details["open_action"]
            qty = strategy_details["open_qty"]

            if ticker is None or len(ticker.domBids) == 0 or len(ticker.domAsks) == 0:
                print(f"************ Issue getting orderbook tickers *************** ")
                alert(True)
                ib.waitOnUpdate()
                continue

            if strategy_details["open_ref"] == "bid":
                price_ref = ticker.domBids[0].price
            elif strategy_details["open_ref"] == "ask":
                price_ref = ticker.domAsks[0].price
            elif strategy_details["open_ref"] == "mid":
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
            else:
                raise Exception("Not implemented")

            lmtPrice = (
                price_ref
                + strategy_details["open_ticks"] * strategy_details["tick_increment"]
            )

            if strategy_details["open_type"] == "LIMIT":
                open_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception("Not implemented")

            print(f"Placing order for open_trade {open_order} ")

            if LIVE:
                open_trade = ib.placeOrder(NQM4, open_order)
                ib.sleep(1)

                trade = open_trade
                push_notifications(f"OPEN ORDER PLACED :: #{trade.order.permId} {trade.orderStatus.status} {trade.contract.symbol} {trade.order.action} {trade.orderStatus.filled}/{trade.orderStatus.remaining} @ {trade.order.lmtPrice}", push)

            else:
                print(f"[NOT LIVE] OPEN ORDER PLACED :: {open_order}")
            
            open_order_ts = datetime.datetime.now()
        

        if open_trade is not None:
            if (
                open_trade.orderStatus.status == "Inactive"
                or open_trade.orderStatus.status == "Cancelled"
            ) and close_trade is None:                
                push_notifications(f"OPEN TRADE CANCELLED:: {open_trade.order}", push)
                open_trade = None
            elif open_trade.orderStatus.status == "Submitted" and close_trade is None:
                print(
                    f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})"
                )

                if (
                    open_order_ts is None
                    or datetime.datetime.now() - open_order_ts
                    > datetime.timedelta(seconds=strategy_details["pause_seconds"])
                ):
                    print(f"OPEN Trade submitted, but not filled. Modifying order limit price: {open_trade.order.lmtPrice}")

                    # find the price of opening the trade
                    if strategy_details["open_ref"] == "bid":
                        price_ref = ticker.domBids[0].price
                    elif strategy_details["open_ref"] == "ask":
                        price_ref = ticker.domAsks[0].price
                    elif strategy_details["open_ref"] == "mid":
                        price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
                    else:
                        raise Exception("Not implemented")

                    lmtPrice = (
                        price_ref
                        + strategy_details["open_ticks"] * strategy_details["tick_increment"]
                    )

                    open_trade.order.lmtPrice = lmtPrice

                    print(f"Modifying order: {open_trade.order}")

                    if LIVE:
                        open_trade = ib.placeOrder(NQM4, open_trade.order)
                        ib.sleep(1)
                        push_notifications(f"MODIFIED ORDER PLACED:: {open_trade.order}", push)
                    else:
                        print(f"[NOT LIVE] MODIFIED ORDER PLACED:: {open_trade.order}")

            elif open_trade.orderStatus.status == "Filled" and close_trade is None:
                print(f"Filled on open_trade {open_trade.orderStatus.status} {open_trade.orderStatus.avgFillPrice} {open_trade.orderStatus.status}")

                action = strategy_details["close_action"]
                qty = strategy_details["close_qty"]

                # find the price to close the trade
                if strategy_details["close_ref"] == "open_fill":
                    price_ref = open_trade.fills[0].execution.avgPrice
                elif strategy_details["close_ref"] == "bid":
                    price_ref = ticker.domBids[0].price
                elif strategy_details["close_ref"] == "ask":
                    price_ref = ticker.domAsks[0].price
                elif strategy_details["close_ref"] == "mid":
                    price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price) / 2
                else:
                    raise Exception("Not implemented")

                lmtPrice = (
                    price_ref
                    + strategy_details["close_ticks"]
                    * strategy_details["tick_increment"]
                )

                if strategy_details["close_type"] == "LIMIT":
                    close_order = LimitOrder(
                        action=action,
                        totalQuantity=qty,
                        lmtPrice=lmtPrice,
                        account="U10394496",
                    )
                else:
                    raise Exception("Not implemented")

                print("Placing close_order {close_order}")

                if LIVE:
                    close_trade = ib.placeOrder(NQM4, close_order)
                    ib.sleep(1)
                    push_notifications(f"OPEN ORDER FILLED:: {open_trade.order}", push)
                    push_notifications(f"CLOSE ORDER PLACED:: {close_trade.order}", push)
                    alert(False)
                else:
                    print(f"[NOT LIVE] OPEN ORDER FILLED:: {open_trade.order}")
                    # print(f"CLOSE ORDER PLACED:: {close_trade.order}")
    
        if close_trade is not None:
            if close_trade.orderStatus.status == "Filled":
                print(f"Close trade filled @ {close_trade.orderStatus.avgFillPrice}")
                print_line()
                push_notifications(f"CLOSE TRADE FILLED:: {close_trade.order}", push)
                alert(True)
            
                open_trade = None
                close_trade = None
                pause = True
            elif (
                close_trade.orderStatus.status == "Inactive"
                or close_trade.orderStatus.status == "Cancelled"
            ):                
                push_notifications(f"CLOSE TRADE CANCELLED:: {close_trade.order}", push)
                print("Order cancelled/inactive. Resetting close_trade to None")
                close_trade = None



SELL_SCALP = {
    "strategy": "SELL TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "BUY",
    "close_ref": "open_fill",
    "close_ticks": -10,
    "pause_seconds": 90,
}

BUY_SCALP = {
    "strategy": "BUY TO OPEN SCALP",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "ask",
    "open_ticks": -10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action": "SELL",
    "close_ref": "open_fill",
    "close_ticks": 10,
    "pause_seconds": 70,
}

run_ss_2(strategy_details=SELL_SCALP, open_permid=open_permid, close_permid = None, push = False)


[21:29:38] {'strategy': 'SELL TO OPEN SCALP', 'contract': 'NQM2024', 'tick_increment': 0.25, 'open_qty': 1, 'open_type': 'LIMIT', 'open_action': 'SELL', 'open_ref': 'ask', 'open_ticks': 10, 'close_qty': 1, 'close_type': 'LIMIT', 'close_action': 'BUY', 'close_ref': 'open_fill', 'close_ticks': -10, 'pause_seconds': 90} / 1533812277 / None
[21:29:38] OPEN TRADE:: Trade(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), order=Order(permId=1533812277, action='SELL', orderType='LMT', lmtPrice=18582.5, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 17:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='N

KeyboardInterrupt: 

In [ ]:
trades = parse_ibrecords(ib.trades())
submittedTrades = [o for o in trades if o["status"] == "Submitted"]

print_ibrecords_table(
    submittedTrades,
    ["orderId", "permId", "status", "orderType", "action", "lmtPrice", "remaining"],
)


In [ ]:
# openTrades = parse_ibrecords(ib.openTrades())
accountSummary = ib.accountSummary()
accountValues = ib.accountValues()

# openTrades = parse_ibrecords(ib.openTrades())
# # util.df([d for d in openTrades])

# print_ibrecords_table(openTrades)

# openOrders = parse_ibrecords(ib.openOrders())
# util.df([d for d in openOrders])

# trades = parse_ibrecords(ib.trades())
# util.df([d for d in trades])

# orders = parse_ibrecords(ib.orders())
# util.df([d for d in orders])


openOrders = parse_ibrecords(ib.openOrders())
openOrders


In [ ]:
from datetime import datetime

input_string = "20240930 16:00:00 Central Standard Time"
date_format = "%Y%m%d %H:%M:%S %Z"

try:
    datetime_object = datetime.strptime(input_string, date_format)
    print(datetime_object)
except ValueError:
    print(f"Incorrect date format.")


In [ ]:
clear_output()
o = ib.orders()[0]
o = util.dataclassNonDefaults(o)
o.pop("softDollarTier", None)
o.pop("shareholder", None)
o.pop("autoCancelDate", None)
o.pop("auxPrice", None)

data, count = supabase.table("orders").upsert(o).execute()

util.tree(o)


In [ ]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')
